In [ ]:
import os
import os.path

import math

import pandas as pd

import numpy as np

import sklearn.preprocessing
import sklearn.pipeline
import sklearn.impute
import sklearn.compose
import sklearn.model_selection
import sklearn.linear_model
import sklearn.metrics

import seaborn as sns

import geopy.geocoders
import geopy.extra.rate_limiter

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_csv(os.path.join(os.getcwd(), 'NH_ProviderInfo_Jan2022_with_coords.csv'))
df.head()

In [ ]:
df.info()

In [ ]:
cat_cols = ['Provider City',
            'Provider State',
            'Provider Zip Code',
            'Provider SSA County Code',
            'Ownership Type',
            'Provider Type',
            'Provider Resides in Hospital',
            'Continuing Care Retirement Community',
            'Special Focus Status',
            'Abuse Icon',
            'Most Recent Health Inspection More Than 2 Years Ago',
            'Provider Changed Ownership in Last 12 Months',
            'With a Resident and Family Council',
            'Automatic Sprinkler Systems in All Required Areas',
            'Long-Stay QM Rating Footnote',
            'Short-Stay QM Rating Footnote',
            'Staffing Rating Footnote',
            'RN Staffing Rating Footnote',
            'Reported Staffing Footnote',
            'Physical Therapist Staffing Footnote',
            'Total nursing staff turnover footnote',
            'Registered Nurse turnover footnote',
            'Administrator turnover footnote']

num_cols = ['Number of Certified Beds',
            'Average Number of Residents per Day',
            'Overall Rating',
            'Health Inspection Rating',
            'QM Rating',
            'Long-Stay QM Rating',
            'Short-Stay QM Rating',
            'Staffing Rating',
            'RN Staffing Rating',
            'Reported Nurse Aide Staffing Hours per Resident per Day',
            'Reported LPN Staffing Hours per Resident per Day',
            'Reported RN Staffing Hours per Resident per Day',
            'Reported Licensed Staffing Hours per Resident per Day',
            'Reported Total Nurse Staffing Hours per Resident per Day',
            'Total number of nurse staff hours per resident per day on the weekend',
            'Registered Nurse hours per resident per day on the weekend',
            'Reported Physical Therapist Staffing Hours per Resident Per Day',
            'Total nursing staff turnover',
            'Registered Nurse turnover',
            'Number of administrators who have left the nursing home',
            'Case-Mix Nurse Aide Staffing Hours per Resident per Day',
            'Case-Mix LPN Staffing Hours per Resident per Day',
            'Case-Mix RN Staffing Hours per Resident per Day',
            'Case-Mix Total Nurse Staffing Hours per Resident per Day',
            'Adjusted Nurse Aide Staffing Hours per Resident per Day',
            'Adjusted LPN Staffing Hours per Resident per Day',
            'Adjusted RN Staffing Hours per Resident per Day',
            'Adjusted Total Nurse Staffing Hours per Resident per Day',
            'Rating Cycle 1 Total Number of Health Deficiencies',
            'Rating Cycle 1 Number of Standard Health Deficiencies',
            'Rating Cycle 1 Number of Complaint Health Deficiencies',
            'Rating Cycle 1 Health Deficiency Score',
            'Rating Cycle 1 Number of Health Revisits',
            'Rating Cycle 1 Health Revisit Score',
            'Rating Cycle 1 Total Health Score',
            'Rating Cycle 2 Total Number of Health Deficiencies',
            'Rating Cycle 2 Number of Standard Health Deficiencies',
            'Rating Cycle 2 Number of Complaint Health Deficiencies',
            'Rating Cycle 2 Health Deficiency Score',
            'Rating Cycle 2 Number of Health Revisits',
            'Rating Cycle 2 Health Revisit Score',
            'Rating Cycle 2 Total Health Score',
            'Rating Cycle 3 Total Number of Health Deficiencies',
            'Rating Cycle 3 Number of Standard Health Deficiencies',
            'Rating Cycle 3 Number of Complaint Health Deficiencies',
            'Rating Cycle 3 Health Deficiency Score',
            'Rating Cycle 3 Number of Health Revisits',
            'Rating Cycle 3 Health Revisit Score',
            'Rating Cycle 3 Total Health Score',
            'Total Weighted Health Survey Score',
            'Number of Facility Reported Incidents',
            'Number of Substantiated Complaints',
            'Number of Citations from Infection Control Inspections']

target_column = 'Total Amount of Fines in Dollars'

In [ ]:
binner = sklearn.compose.ColumnTransformer([
    ("kbins", sklearn.preprocessing.KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile'), [target_column])
])

bins = binner.fit_transform(df)
sns.histplot(bins)

In [ ]:
categorical_pipeline = sklearn.pipeline.make_pipeline(
    sklearn.preprocessing.OneHotEncoder()
)

numeric_pipeline = sklearn.pipeline.make_pipeline(
    sklearn.impute.SimpleImputer(strategy='median'),
    sklearn.preprocessing.StandardScaler()
)

full_pipeline = sklearn.compose.ColumnTransformer([
    ("categorical", categorical_pipeline, cat_cols),
    ("numeric", numeric_pipeline, num_cols)
])

In [ ]:
features_matrix = full_pipeline.fit_transform(df)
features_matrix.shape

In [ ]:
training_features, testing_features, training_targets, testing_targets = sklearn.model_selection.train_test_split(features_matrix, df[target_column], test_size=0.25, stratify=bins)

In [ ]:
model = sklearn.linear_model.SGDRegressor()
model.fit(training_features, training_targets)

In [ ]:
predictions = model.predict(testing_features)

In [ ]:
error = math.sqrt(sklearn.metrics.mean_squared_error(testing_targets, predictions))
error

In [ ]:
df['Full Address'] = df['Provider Address'] + ", " + df['Provider City'] + ", " + df['Provider State'] + " " + df['Provider Zip Code'].astype(str)
df.head()

In [ ]:
address = df.iloc[0]['Location']

In [ ]:
geolocator = geopy.geocoders.Nominatim(user_agent='jacks-ml-project')
geocode_rate_limited = geopy.extra.rate_limiter.RateLimiter(geolocator.geocode, min_delay_seconds=1)

This next cell takes hours to run, as it calls a web service for each row in the dataset and is rate-limited to 1/sec. This has already been run and is stored in `NH_ProviderInfo_Jan2022_with_coords.csv`. 

In [ ]:
# df['geo_loc'] = df['Full Address'].apply(geocode_rate_limited)
# df['geo_point'] = df['geo_loc'].apply(lambda x: tuple(x.point)[0:2] if x else (np.nan, np.nan))
# df[['lat', 'lon']] = pd.DataFrame(df['geo_point'].tolist(), index=df.index)
# df = df.drop(columns=['geo_loc', 'geo_point'])

In [ ]:
df[df['lat'].isnull()]

In [ ]:
missing_coords = df[df['lat'].isnull()]
bing_geolocator = geopy.geocoders.Bing('<API key here>')
bing_geocode_rate_limited = geopy.extra.rate_limiter.RateLimiter(bing_geolocator.geocode, min_delay_seconds=1)

missing_coords['geo_loc'] = missing_coords['Full Address'].apply(bing_geocode_rate_limited)
missing_coords['geo_point'] = missing_coords['geo_loc'].apply(lambda x: tuple(x.point)[0:2] if x else (np.nan, np.nan))
missing_coords[['lat', 'lon']] = pd.DataFrame(missing_coords['geo_point'].tolist(), index=missing_coords.index)
missing_coords = missing_coords.drop(columns=['geo_loc', 'geo_point'])

In [ ]:
missing_coords